In [166]:
import pandas as pd
import numpy as np
# utm
# pip install utm
import utm
import re
from glob import glob

pd.options.mode.chained_assignment = None

## Lecture de données CSV

### Trafic Moyen Journalier Annuel

In [167]:
data = pd.read_csv('./data/input/TMJA_2007.csv',delimiter='\t' )
data.columns

Index(['dateReferentiel', 'route', 'longueur', 'prD', 'depPrD',
       'concessionPrD', 'absD', 'cumulD', 'xD', 'yD', 'zD', 'prF', 'depPrF',
       'concessionPrF', 'absF', 'cumulF', 'xF', 'yF', 'zF',
       'anneeMesureTrafic', 'typeComptageTrafic', 'TMJA', 'RatioPL'],
      dtype='object')

In [168]:
def raccourcir(x):
    return x[0:6]

def data_cleaning(url):

    data = pd.read_csv(url,delimiter='\t' )
    data_clean = data[['anneeMesureTrafic','TMJA', 'xD', 'yD','xF', 'yF','route', 'longueur']].dropna()

    year=data_clean['anneeMesureTrafic'].iloc[0]

    for i in range(len(data_clean)):
        data_clean['xD'].iloc[i]=raccourcir(data_clean['xD'].iloc[i])
        data_clean['yD'].iloc[i]=raccourcir(data_clean['yD'].iloc[i])
        data_clean['xF'].iloc[i]=raccourcir(data_clean['xF'].iloc[i])
        data_clean['yF'].iloc[i]=raccourcir(data_clean['yF'].iloc[i])
    
    data_clean['xD']=data_clean['xD'].astype('int')
    data_clean['yD']=data_clean['yD'].astype('int')
    data_clean['xF']=data_clean['xF'].astype('int')
    data_clean['yF']=data_clean['yF'].astype('int')

    data_clean['coordGPS_D_lon'] = 0
    data_clean['coordGPS_D_lat'] = 0
    data_clean['coordGPS_F_lon'] = 0
    data_clean['coordGPS_F_lat'] = 0

    for i in range(len(data_clean)):
        data_clean['coordGPS_D_lat'].iloc[i]=[utm.to_latlon(easting=data_clean['xD'].iloc[i],
                                                                northing=data_clean['yD'].iloc[i],
                                                                northern=True,
                                                                zone_number=31)][0][0]
        data_clean['coordGPS_D_lon'].iloc[i]=[utm.to_latlon(easting=data_clean['xD'].iloc[i],
                                                                northing=data_clean['yD'].iloc[i],
                                                                northern=True,
                                                                zone_number=31)][0][1]                                                              
        data_clean['coordGPS_F_lat'].iloc[i]=[utm.to_latlon(easting=data_clean['xF'].iloc[i],
                                                                northing=data_clean['yF'].iloc[i],
                                                                northern=True,
                                                                zone_number=31)][0][0]
        data_clean['coordGPS_F_lon'].iloc[i]=[utm.to_latlon(easting=data_clean['xF'].iloc[i],
                                                                northing=data_clean['yF'].iloc[i],
                                                                northern=True,
                                                                zone_number=31)][0][1]
    data_clean.to_csv(f"./data/clean/TMJA_{int(year)}_clean.csv")

In [169]:
liste_csv = glob('./data/input/*.csv')
for i,f in enumerate(liste_csv):
    data_cleaning(f)

#### Chaque section de route est définie par :

son nom (route)

sa longueur

##### ses coordonnées de début (D) et de fin (F) exprimées en coordonnées (X, Y, Z) ou par un système de repérage routier composé de 4 attributs :

pr = Point de repère routier

depPr = département où se situe le PR

concessionPr = indique si le PR se trouve sur une section concédée (C) ou non (N)

abs = abscisse ou distance (en mètres) séparant le point du PR auquel il se rattache

##### D'autres attributs complètent ces attributs de localisation :

anneeMesureTrafic = année pendant laquelle ont été effectuées les mesures

TypeComptageTrafic = indique si le comptage des véhicules a été effectué à partir d'un système placé en permanence sur la route ou s'il s'agit d'un relevé à l'aide d'un système de comptage temporaire

TMJA = trafic moyen journalier annuel

RatioPL = pourcentage de poids-lourds entrant dans la composition du TMJA

#### Concaténation des différentes années

In [181]:
TMJA_clean = pd.DataFrame()

liste_csv_clean = glob('./data/clean/*.csv')

for i,f in enumerate(liste_csv_clean):
    df = pd.read_csv(f,delimiter=',')
    df_ =pd.concat([TMJA_clean,df], axis=0)
    TMJA_clean = df_

TMJA_clean = TMJA_clean.drop(columns='Unnamed: 0', axis=1)

In [183]:
TMJA_clean.to_csv("./data/clean/TMJA_2007_2018_clean.csv")